In [ ]:
import cv2
from skimage.measure import find_contours, approximate_polygon,subdivide_polygon
import numpy as np
import matplotlib.pyplot as plt
import PIL
import operator

PIL.Image.MAX_IMAGE_PIXELS = 1000000000 #933120000

In [ ]:

def index_W(contour_copy,p1,p2,last_ind):
    it,ind1,ind2 = 1, 0, 0
    for i in range(last_ind,len(contour_copy)):
        if it == 1 and contour_copy[i][0] == p1[0] and contour_copy[i][1] == p1[1]:
            ind1 = i
            it = 2

        elif it == 2 and contour_copy[i][0] == p2[0] and contour_copy[i][1] == p2[1]:
            ind2 = i
            it = 3

        elif it == 3:
            break
    return ind1, ind2

def detect_RoI(phase1_output,intrested_distance,resolution,mosaic_rgb_addr,add_to_save):
    #****************************************
    s = int(intrested_distance/resolution)+1

    orig_img = cv2.imread(mosaic_rgb_addr)
    fragmentary = np.ones((orig_img.shape[0],orig_img.shape[1],3))*255
    fragmentary = fragmentary.astype(np.uint8)

    predict = cv2.imread(phase1_output)
    p = cv2.imread(phase1_output)[:,:,2]
    predict[:,:,0],predict[:,:,1],predict[:,:,2] = p, p , p

    orig_img2 = orig_img - (orig_img *predict) + (predict*255)
    del predict

    #*****************************************

    del orig_img
    color = ['-g','-r','-m']
    c = 0
    ind1, ind2 = 0,0
    all_rectangles = []

    for contour in find_contours(p*50, 40):

        last_ind = 0
        coords = approximate_polygon(contour, tolerance=1)

        if len(coords) > 0 :
            for n in range(len(coords)-1):

                #find index of corners in contour:
                p1, p2 = coords[n], coords[n+1]

                ind1, ind2 = index_W(contour,p1,p2,last_ind)
                last_ind = ind2

                # find border:
                border = contour[min(ind1,ind2) : max(ind1,ind2)]
                min_x, max_x = min(border,key=operator.itemgetter(0))[0], max(border,key=operator.itemgetter(0))[0]
                min_y, max_y = min(border,key=operator.itemgetter(1))[1], max(border,key=operator.itemgetter(1))[1]

                rectangle = np.array([ [min_x-s,min_y-s], [min_x-s,max_y+s], [max_x+s,max_y+s], [max_x+s,min_y-s], [min_x-s,min_y-s] ])


                # save extended rectangles
                (min_x,min_y,max_x,max_y ) = np.round((min_x,min_y,max_x,max_y))
                min_x,min_y, max_x,max_y = int(min_x),int(min_y), int(max_x),int(max_y)
                fragmentary[min_x-s:max_x+s , min_y-s:max_y+s,:] = orig_img2[min_x-s:max_x+s , min_y-s:max_y+s,:]


                if c < 2:
                    c += 1
                else:
                    c = 0


    fragmentary = fragmentary

    cv2.imwrite(add_to_save, fragmentary)
    print('RoI is saved')


In [ ]:
def save_patches (RoI,anno, addr_to_save,addr_anno,patch_size):
    img = cv2.imread(RoI)
    anno2= cv2.imread(anno)

    (patchx,patchy) = patch_size
    x0,y0 = 0,0
    line = 1
    x1 = x0+ patchx
    while x0+patchx <= img.shape[0]:
        y0,y1 = 0, 0
        x1 = x0+patchx
        count = 0
        while y0+patchy <= img.shape[1]:
            count+=1
            y1 = y0+patchy

            crop = img[x0:x1,y0:y1,:]
            cropanno = anno2[x0:x1,y0:y1,:]
            if np.min(crop) < 200:

                cv2.imwrite(addr_to_save+'barra '+str(line)+' '+str(count)+' .png', crop)
                cv2.imwrite(addr_anno+'barra '+str(line)+' '+str(count)+' .png', cropanno)

            if y1<img.shape[1] and y1+patchy > img.shape[1] :
                y0 = img.shape[1]-patchy

            else:
                y0 = y1

        line+=1
        if x1<img.shape[0] and x1+patchx > img.shape[0] :
                x0 = img.shape[0]-patchx
        else:
            x0 = x1
